<a href="https://colab.research.google.com/github/flaviopradoaquino/TCC_MBA_USP_ESALQ/blob/main/TCC_USP_Teste_Estacionaridade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalação e Importação das Bibliotecas**

In [ ]:
!pip install -q yfinance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import statsmodels.tsa.stattools
import plotly.express as px
from datetime import date, timedelta

# **Construção do Dataset**

In [ ]:
dateparse = lambda dates: datetime.datetime.strptime(dates, '%Y-%m-%d')

In [ ]:
from datetime import datetime 
dt_atual = date.today()
td = timedelta(1095)
dt_final = datetime(2023, 3, 3)

In [ ]:
### Data de referência menos 1095 dias
df_teste_est = yf.download("BOVA11.SA", start=(dt_final-td))['Close']

### Data atual menos 1095 dias
# df = yf.download("BOVA11.SA", start=(dt_atual-td))['Close']

### Intervalo de datas fixas
# df = yf.download("BOVA11.SA", start="2020-03-03", end="2023-03-03")['Close']

In [ ]:
df_teste_est

# **Teste de Estacionaridade KPSS (Kwiatkowski-Phillips-Schmidt-Shin)**

**Ho = não é estacionário: estatística do teste > valor crítico**

**Ha = é estacionário: estatística do teste < valor crítico**

In [ ]:
kpss_01 = statsmodels.tsa.stattools.kpss(df_teste_est)
print('Estatítica do teste: {:.4f}'.format(kpss_01[0]))
print('p_valor: {:.4f}'.format(kpss_01[1]))
print('número de lags: {}'.format(kpss_01[2]))
print('Valores Críticos:')
for chave, valor in kpss_01[3].items():
   print('{}: {:.4f}'.format(chave, valor))

# **Teste de Estacionaridade DFuller (Dickey Fuller)**

**Ho = não é estacionário: estatística do teste > valor crítico**

**Ha = é estacionário: estatística do teste < valor crítico**

In [ ]:
dfuller_01 = statsmodels.tsa.stattools.adfuller(df_teste_est)
print('Estatítica do teste: {:.4f}'.format(dfuller_01[0]))
print('p_valor: {:.8f}'.format(dfuller_01[1]))
print('Valores Críticos:')
for chave, valor in dfuller_01[4].items():
   print('{}: {:.4f}'.format(chave, valor))

# **Excluindo Outliers para o Teste de Estacionaridade**

In [ ]:
df = df_teste_est.reset_index()

In [ ]:
df

In [ ]:
df_bova_temp = df[['Date', 'Close']].rename(columns = {'Date': 'ds', 'Close': 'y'})

In [ ]:
df_bova_temp

In [ ]:
df_bova = pd.DataFrame({
  "ds": df_bova_temp['ds'].dt.date,
  "y":df_bova_temp['y']
})

In [ ]:
dados = df_bova['y']
dados.describe()

In [ ]:
boxplot_bova = px.box(dados, y="y")
boxplot_bova.show()

In [ ]:
q1 = np.quantile(dados, .25, method="midpoint")
q2 = np.quantile(dados, .5, method="midpoint")
q3 = np.quantile(dados, .75, method="midpoint")
IIQ = q3 - q1

In [ ]:
### Limite inferior do boxplot
limite_inferior = q1 - 1.5 * IIQ
limite_inferior

In [ ]:
### Limite superior do boxplot
limite_superior = q3 + 1.5 * IIQ
limite_superior

In [ ]:
df_bova_lim_inf = df_bova[df_bova['y']>limite_inferior]
df_bova_clean = df_bova_lim_inf[df_bova_lim_inf['y']<limite_superior]
df_bova = df_bova_clean
df_bova

In [ ]:
df_teste_estacionaridade = pd.Series(df_bova['y'].values, index = df_bova['ds'])

# **Teste de Estacionaridade KPSS (Kwiatkowski-Phillips-Schmidt-Shin)**

**Ho = não é estacionário: estatística do teste > valor crítico**

**Ha = é estacionário: estatística do teste < valor crítico**

In [ ]:
kpss_02 = statsmodels.tsa.stattools.kpss(df_teste_estacionaridade)
print('Estatítica do teste: {:.4f}'.format(kpss_02[0]))
print('p_valor: {:.4f}'.format(kpss_02[1]))
print('número de lags: {}'.format(kpss_02[2]))
print('Valores Críticos:')
for chave, valor in kpss_02[3].items():
   print('{}: {:.4f}'.format(chave, valor))

# **Teste de Estacionaridade DFuller (Dickey Fuller)**

**Ho = não é estacionário: estatística do teste > valor crítico**

**Ha = é estacionário: estatística do teste < valor crítico**

In [ ]:
dfuller_02 = statsmodels.tsa.stattools.adfuller(df_teste_estacionaridade)
print('Estatítica do teste: {:.4f}'.format(dfuller_02[0]))
print('p_valor: {:.8f}'.format(dfuller_02[1]))
print('Valores Críticos:')
for chave, valor in dfuller_02[4].items():
   print('{}: {:.4f}'.format(chave, valor))